# Gesture Recognition - Model 3
#### Conv3D Model with comprehensive preprocessing (using OpenCV library - Resize and Data augmetation with Affine transformation, Horizontal Flip, Cropping and Resizing,  Usage of Optimizer, ReduceLROnPlateau to reduce learning rate, and  experimenting with various sizes of batch sizes and Epochs)
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator and Preprocessing Functions
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 15  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
# Function to initialize all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [7]:
# All image preprocessing is moved to this function to make it moduler
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = initialize_batch_data(batch_size)
    
    # Build an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # Build an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [8]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)           
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Final Model

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
model.add(Conv3D( 8, (3,3,3), activation='relu', input_shape=input_shape ))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(16, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(32, (3,3,3), activation='relu'))
model.add(Conv3D(32, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(64, (2,2,2), activation='relu'))
model.add(Conv3D(64, (2,2,2), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 9, 26, 26, 32)     13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 24, 24, 32)     27680     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 7, 12, 12, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train Epoch 1/20; batch size = 10

67/67 [==============================] - 132s 2s/step - loss: 13.0938 - categorical_accuracy: 0.1851 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-13.11820-0.18351-13.53920-0.16000.h5
Epoch 2/20
67/67 [==============================] - 13s 200ms/step - loss: 13.3917 - categorical_accuracy: 0.1692 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-13.39165-0.16915-13.53920-0.16000.h5
Epoch 3/20
67/67 [==============================] - 14s 216ms/step - loss: 12.7501 - categorical_accuracy: 0.2090 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-12.75014-0.20896-13.53920-0.16000.h5

Epoch 00003: ReduceLROnPlateau reducing 

In [16]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 7s 209ms/step - loss: 13.9058 - categorical_accuracy: 0.1373 - val_loss: 13.2168 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-13.90581-0.13725-13.21684-0.18000.h5
Epoch 2/20
34/34 [==============================] - 7s 195ms/step - loss: 12.4836 - categorical_accuracy: 0.2255 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-12.48362-0.22549-13.53920-0.16000.h5
Epoch 3/20
34/34 [==============================] - 6s 184ms/step - loss: 12.6416 - categorical_accuracy: 0.2157 - val_loss: 14.8286 - val_categorical_accuracy: 0.0800

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-12.64164-0.21569-14.82865-0.08000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
Epoch 4/20
34/34 [==============================] - 6s 173ms/step -

In [17]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 5s 196ms/step - loss: 11.9134 - categorical_accuracy: 0.2609 - val_loss: 13.2974 - val_categorical_accuracy: 0.1750

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-11.91337-0.26087-13.29743-0.17500.h5
Epoch 2/30
23/23 [==============================] - 4s 191ms/step - loss: 13.0814 - categorical_accuracy: 0.1884 - val_loss: 14.1033 - val_categorical_accuracy: 0.1250

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-13.08135-0.18841-14.10333-0.12500.h5
Epoch 3/30
23/23 [==============================] - 4s 184ms/step - loss: 11.6798 - categorical_accuracy: 0.2754 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-11.67978-0.27536-12.89448-0.20000.h5
Epoch 4/30
23/23 [==============================] - 5s 204ms/step - loss: 12.8478 - categorical_accuracy: 0.2029 - val_loss: 12.4915 - val_categoric

23/23 [==============================] - 4s 187ms/step - loss: 10.5897 - categorical_accuracy: 0.3430 - val_loss: 13.2974 - val_categorical_accuracy: 0.1750

Epoch 00027: saving model to model_init_2020-03-3015_21_49.611254/model-00027-10.58967-0.34300-13.29743-0.17500.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 3.725290475403709e-12.
Epoch 28/30
23/23 [==============================] - 4s 180ms/step - loss: 13.7821 - categorical_accuracy: 0.1449 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00028: saving model to model_init_2020-03-3015_21_49.611254/model-00028-13.78214-0.14493-13.70038-0.15000.h5
Epoch 29/30
23/23 [==============================] - 4s 179ms/step - loss: 13.3149 - categorical_accuracy: 0.1739 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00029: saving model to model_init_2020-03-3015_21_49.611254/model-00029-13.31495-0.17391-12.89448-0.20000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.8626452377018543

In [18]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 3s 193ms/step - loss: 13.0480 - categorical_accuracy: 0.1905 - val_loss: 15.3122 - val_categorical_accuracy: 0.0500

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-13.04798-0.19048-15.31219-0.05000.h5
Epoch 2/30
14/14 [==============================] - 2s 175ms/step - loss: 13.8155 - categorical_accuracy: 0.1429 - val_loss: 15.3122 - val_categorical_accuracy: 0.0500

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-13.81551-0.14286-15.31219-0.05000.h5
Epoch 3/30
14/14 [==============================] - 2s 154ms/step - loss: 14.5830 - categorical_accuracy: 0.0952 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-14.58304-0.09524-12.89448-0.20000.h5
Epoch 4/30
14/14 [==============================] - 2s 167ms/step - loss: 12.2805 - categorical_accuracy: 0.2381 - val_loss: 14.5063 - val_categoric

14/14 [==============================] - 2s 170ms/step - loss: 13.8155 - categorical_accuracy: 0.1429 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00027: saving model to model_init_2020-03-3015_21_49.611254/model-00027-13.81551-0.14286-13.70038-0.15000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.818989489943217e-15.
Epoch 28/30
14/14 [==============================] - 2s 163ms/step - loss: 10.3616 - categorical_accuracy: 0.3571 - val_loss: 14.5063 - val_categorical_accuracy: 0.1000

Epoch 00028: saving model to model_init_2020-03-3015_21_49.611254/model-00028-10.36163-0.35714-14.50629-0.10000.h5
Epoch 29/30
14/14 [==============================] - 2s 160ms/step - loss: 14.1993 - categorical_accuracy: 0.1190 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00029: saving model to model_init_2020-03-3015_21_49.611254/model-00029-14.19927-0.11905-12.89448-0.20000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 9.094947449716085e

In [19]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 2s 209ms/step - loss: 14.1033 - categorical_accuracy: 0.1250 - val_loss: 14.5063 - val_categorical_accuracy: 0.1000

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-14.10333-0.12500-14.50629-0.10000.h5
Epoch 2/30
8/8 [==============================] - 1s 187ms/step - loss: 12.0886 - categorical_accuracy: 0.2500 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-12.08857-0.25000-13.70038-0.15000.h5
Epoch 3/30
8/8 [==============================] - 1s 175ms/step - loss: 12.7602 - categorical_accuracy: 0.2083 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-12.76016-0.20833-12.89448-0.20000.h5
Epoch 4/30
8/8 [==============================] - 2s 189ms/step - loss: 12.0886 - categorical_accuracy: 0.2500 - val_loss: 12.0886 - val_categorical_accur

8/8 [==============================] - 2s 188ms/step - loss: 12.7602 - categorical_accuracy: 0.2083 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00027: saving model to model_init_2020-03-3015_21_49.611254/model-00027-12.76016-0.20833-12.89448-0.20000.h5
Epoch 28/30
8/8 [==============================] - 1s 179ms/step - loss: 14.7749 - categorical_accuracy: 0.0833 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00028: saving model to model_init_2020-03-3015_21_49.611254/model-00028-14.77492-0.08333-13.70038-0.15000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 2.220446154715841e-19.
Epoch 29/30
8/8 [==============================] - 2s 197ms/step - loss: 12.0886 - categorical_accuracy: 0.2500 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00029: saving model to model_init_2020-03-3015_21_49.611254/model-00029-12.08857-0.25000-13.70038-0.15000.h5
Epoch 30/30
8/8 [==============================] - 2s 192ms/step - loss: 12.7602 -

In [20]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 2s 219ms/step - loss: 12.0886 - categorical_accuracy: 0.2500 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-03-3015_21_49.611254/model-00001-12.08857-0.25000-12.89448-0.20000.h5
Epoch 2/20
8/8 [==============================] - 1s 182ms/step - loss: 12.0886 - categorical_accuracy: 0.2500 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-03-3015_21_49.611254/model-00002-12.08857-0.25000-12.89448-0.20000.h5
Epoch 3/20
8/8 [==============================] - 2s 193ms/step - loss: 10.7454 - categorical_accuracy: 0.3333 - val_loss: 13.7004 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2020-03-3015_21_49.611254/model-00003-10.74540-0.33333-13.70038-0.15000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 5.551115386789603e-20.
Epoch 4/20
8/8 [==============================] - 2s 189ms/step - loss: 1